<a href="https://colab.research.google.com/github/KaitlynChou/LIGN-168-Final-Project/blob/master/LIGN168_Final_Project_Kaitlyn_Chou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initiating Charsiu model

In [1]:
# Install libraries
!pip3 install librosa
!pip install pydub
!pip install torch torchvision torchaudio
!pip install datasets transformers
!pip install praatio librosa
!pip install g2p_en g2pM
!pip install sounddevice
!pip install wavio
!pip install scipy
!pip install -U praat-parselmouth
!apt-get update
!apt-get install libsndfile1

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,390 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,168 kB]
Fetched 3,815 kB in 2s (2,349 kB/s)
Reading package lists... Done
Reading

In [2]:
# Download Charsiu library
import os
from os.path import exists, join, expanduser

os.chdir(expanduser("~"))
charsiu_dir = 'charsiu'
if exists(charsiu_dir):
  !rm -rf /root/charsiu
if not exists(charsiu_dir):
  ! git clone -b development https://github.com/lingjzhu/$charsiu_dir
  ! cd charsiu && git checkout && cd -

os.chdir(charsiu_dir)

Cloning into 'charsiu'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 686 (delta 110), reused 79 (delta 79), pack-reused 555
Receiving objects: 100% (686/686), 1.20 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (364/364), done.
Your branch is up to date with 'origin/development'.
/root


In [3]:
# Import libraries for Charsiu
import sys
import torch
from itertools import groupby

sys.path.insert(0,'src')

# Import pretrained textless alignment model
from Charsiu import charsiu_predictive_aligner

# Initialize model
charsiu = charsiu_predictive_aligner(aligner='charsiu/en_w2v2_fc_10ms')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at charsiu/en_w2v2_fc_10ms were not used when initializing Wa

In [4]:
# Connect to Google Drive to access test files
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Creating the main algorithm

In [5]:
# Import libraries
import librosa
import soundfile as sf
from pydub import AudioSegment
import parselmouth
import parselmouth.praat
import numpy as np
import tempfile

# Defining function for performing Charsiu textless alignment
def textless_align(sound):
  # Resample the wav file to 16000Hz
  data, original_sample_rate = librosa.load(sound, sr=None)
  resampled_data = librosa.resample(data, orig_sr=original_sample_rate, target_sr=16000)

  # Write to a temporary file so the original wav isn't changed
  with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as temp_file:
        temp_path = temp_file.name
  sf.write(temp_path, resampled_data, 16000)

  # Perform textless alignment using Charsiu
  alignment = charsiu.align(audio=temp_path)
  return alignment

# Defining main function
def replace_vowels(sound, vowel_to_change, new_vowel):
  path_to_script = '/content/drive/MyDrive/Colab Notebooks/plugin_VocalToolkit/changeformants.praat'

  # Create an alignment for the given sound
  alignment = textless_align(sound)
  print(alignment)

  # Turn the speech into a parselmouth Sound object
  audio = parselmouth.Sound(sound)

  # Downsample for better resynthesis results
  audio = audio.resample(16000)

  # Retrieve duration for concatenation later
  duration = audio.get_total_duration()

  # Go through the third item of each tuple in alignment, which contains its phoneme label.
  for i in range(0, len(alignment)-1):
    if alignment[i][2]==vowel_to_change: # Check if the label we are on matches the vowel that we want to change

      # Extract segment containing the vowel using starting and ending times from alignment
      start_time = alignment[i][0]
      end_time = alignment[i][1]
      extracted_vowel = audio.extract_part(from_time=start_time, to_time=end_time)

      # Change formants with Praat script depending on the desired new vowel
      if new_vowel=='IH':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 360, 2187, 2830, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='EH':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 581, 1840, 2429, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='EE':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 434, 2148, 2763, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='IY':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 294, 2343, 3251, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='AE':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 766, 1782, 2398, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='AH':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 707, 1354, 2289, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='AA':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 781, 1065, 2158, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='UH':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 334, 910, 2300, 0, 0, 5000, 1, 1, 0)[0]
      elif new_vowel=='UW':
        modified_vowel = parselmouth.praat.run_file(extracted_vowel, path_to_script, 295, 750, 2342, 0, 0, 5000, 1, 1, 0)[0]
      else:
        modified_vowel = extracted_vowel

      # Put modified_vowel back with the unmodified portions of the audio
      temp = parselmouth.Sound.concatenate([audio.extract_part(from_time=0, to_time=start_time), modified_vowel, audio.extract_part(from_time=end_time, to_time=duration)])
      audio = temp

  # Upsample so final product sounds more clear
  audio = audio.resample(44100)

  # After every instance of the desired vowel is modified,
  # turn audio (currently a Sound object) into AudioSegment object so it can be played directly
  audio_data = np.array(audio.values.T)
  sample_rate = audio.sampling_frequency
  max_amplitude = np.iinfo(np.int16).max
  normalized_audio_data = (audio_data / np.max(np.abs(audio_data))) * max_amplitude
  audio_bytes = normalized_audio_data.astype(np.int16).tobytes()
  final_sound = AudioSegment(data=audio_bytes, sample_width=2, frame_rate=sample_rate, channels=1)

  return final_sound

# Testing with Google TTS Voice

The vowels in Charsiu are transcribed using the ARPAbet. They correspond to IPA symbols as follows:
*   AE: /æ/
*   EH: /ɛ/
*   IH: /ɪ/
*   IY: /i/
*   AA: /ɑ/
*   AH: /ʌ/ or /ə/
*   UH: /ʊ/
*   UW: /u/

## 1. /ɛ/ EH to /ɪ/ IH

In [6]:
# ORIGINAL: When I win, I will get a pen.
import IPython.display as ipd
ipd.Audio('/content/drive/MyDrive/Colab Notebooks/Test sound files/wheniwin.wav')

In [7]:
# "EH" to "IH"
# When I win, I will get a pen.
test_sound = '/content/drive/MyDrive/Colab Notebooks/Test sound files/wheniwin.wav'
new_test_sound = replace_vowels(test_sound, 'EH', 'IH')
new_test_sound

[(0.0, 0.1, 'W'), (0.1, 0.14, 'EH'), (0.14, 0.2, 'N'), (0.2, 0.34, 'AY'), (0.34, 0.51, 'W'), (0.51, 0.68, 'IH'), (0.68, 0.85, 'N'), (0.85, 1.26, '[SIL]'), (1.26, 1.48, 'AY'), (1.48, 1.58, 'W'), (1.58, 1.65, 'AH'), (1.65, 1.74, 'L'), (1.74, 1.82, 'G'), (1.82, 1.9, 'EH'), (1.9, 1.95, 'T'), (1.95, 2.02, 'AH'), (2.02, 2.04, '[SIL]'), (2.04, 2.19, 'P'), (2.19, 2.36, 'EH'), (2.36, 2.5, 'N'), (2.5, 2.53, '[SIL]')]


In [8]:
# ORIGINAL: How many hens are in the den?
ipd.Audio('/content/drive/MyDrive/Colab Notebooks/Test sound files/hensintheden.wav')

In [9]:
# "EH" to "IH"
# How many hens are in the den?
test_sound_2 = '/content/drive/MyDrive/Colab Notebooks/Test sound files/hensintheden.wav'
new_test_sound_2 = replace_vowels(test_sound_2, 'EH', 'IH')
new_test_sound_2

[(0.0, 0.11, 'HH'), (0.11, 0.22, 'AW'), (0.22, 0.33, 'M'), (0.33, 0.38, 'EH'), (0.38, 0.44, 'N'), (0.44, 0.54, 'IY'), (0.54, 0.75, 'HH'), (0.75, 0.92, 'AE'), (0.92, 1.05, 'N'), (1.05, 1.06, 'D'), (1.06, 1.2, 'Z'), (1.2, 1.28, '[SIL]'), (1.28, 1.4, 'AA'), (1.4, 1.54, 'R'), (1.54, 1.59, 'IH'), (1.59, 1.68, 'N'), (1.68, 1.73, 'DH'), (1.73, 1.81, 'AH'), (1.81, 1.88, 'D'), (1.88, 2.08, 'EH'), (2.08, 2.26, 'N'), (2.26, 2.31, '[SIL]')]


## 2. /æ/ AE to /e/ EE

In [10]:
# ORIGINAL: Put the cat in the bag.
ipd.Audio('/content/drive/MyDrive/Colab Notebooks/Test sound files/catinthebag.wav')

In [11]:
# "AE" to "EE"
# Put the cat in the bag.
test_sound_3 = '/content/drive/MyDrive/Colab Notebooks/Test sound files/catinthebag.wav'
new_test_sound_3 = replace_vowels(test_sound_3, 'AE', 'EE')
new_test_sound_3

[(0.0, 0.02, '[SIL]'), (0.02, 0.09, 'P'), (0.09, 0.17, 'UH'), (0.17, 0.21, 'T'), (0.21, 0.26, 'DH'), (0.26, 0.31, 'AH'), (0.31, 0.46, 'K'), (0.46, 0.64, 'AE'), (0.64, 0.71, 'T'), (0.71, 0.88, 'IH'), (0.88, 0.96, 'N'), (0.96, 1.0, 'DH'), (1.0, 1.05, 'AH'), (1.05, 1.14, 'B'), (1.14, 1.48, 'AE'), (1.48, 1.58, 'G'), (1.58, 1.63, '[SIL]')]


In [12]:
# ORIGINAL: The bag is full of cats.
ipd.Audio('/content/drive/MyDrive/Colab Notebooks/Test sound files/fullofcats.wav')

In [13]:
# "AE" to "EE"
# The bag is full of cats.
test_sound_4 = '/content/drive/MyDrive/Colab Notebooks/Test sound files/fullofcats.wav'
new_test_sound_4 = replace_vowels(test_sound_4, 'AE', 'EE')
new_test_sound_4

[(0.0, 0.01, '[SIL]'), (0.01, 0.06, 'DH'), (0.06, 0.12, 'AH'), (0.12, 0.2, 'B'), (0.2, 0.44, 'AE'), (0.44, 0.5, 'G'), (0.5, 0.56, 'IH'), (0.56, 0.68, 'Z'), (0.68, 0.81, 'F'), (0.81, 0.86, 'UH'), (0.86, 0.98, 'L'), (0.98, 1.02, 'AH'), (1.02, 1.08, 'V'), (1.08, 1.11, '[SIL]'), (1.11, 1.26, 'K'), (1.26, 1.47, 'AE'), (1.47, 1.57, 'T'), (1.57, 1.75, 'S'), (1.75, 1.81, '[SIL]')]


## 3. /u/ UW to /ʌ/ AH

In [14]:
# ORIGINAL: There is a pair of boots in the room.
ipd.Audio('/content/drive/MyDrive/Colab Notebooks/Test sound files/bootsinroom.wav')

In [16]:
test_sound_5 = '/content/drive/MyDrive/Colab Notebooks/Test sound files/bootsinroom.wav'
new_test_sound_5 = replace_vowels(test_sound_5, 'UW', 'AH')
new_test_sound_5

[(0.0, 0.01, '[SIL]'), (0.01, 0.08, 'DH'), (0.08, 0.15, 'EH'), (0.15, 0.25, 'R'), (0.25, 0.3, 'IH'), (0.3, 0.37, 'Z'), (0.37, 0.45, 'AH'), (0.45, 0.61, 'P'), (0.61, 0.71, 'EH'), (0.71, 0.85, 'R'), (0.85, 0.91, 'AH'), (0.91, 0.98, 'V'), (0.98, 0.99, '[SIL]'), (0.99, 1.02, 'V'), (1.02, 1.1, 'B'), (1.1, 1.29, 'UW'), (1.29, 1.36, 'T'), (1.36, 1.52, 'S'), (1.52, 1.74, '[SIL]'), (1.74, 1.88, 'IH'), (1.88, 1.96, 'N'), (1.96, 2.0, 'DH'), (2.0, 2.1, 'AH'), (2.1, 2.25, 'R'), (2.25, 2.42, 'UW'), (2.42, 2.58, 'M'), (2.58, 2.63, '[SIL]')]
